#MultiSig

<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/08_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [ ]:
! pip install eciespy eth_keys cryptos qrcode base58 
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git

# Crear Transacciones de Criptomoneda (Create Cryptocurrency Transactions)

## Esquema (Outline) 

***

 
## Metas (Goals)
***
***
By the end of this notebook you should be able to:
***
Al final de este cuaderno, usted debería ser capaz de:
***
***
- Multisig
-

# Importaciones y Definiciones (Imports and Definitions)

Crear funciones para importar claves y direcciones de criptomonedas.
***
Create functions for importing keys and cryptocurrency addresses.

In [ ]:
import hashlib
import os
import getpass
import ecies
import eth_keys
import cryptos
import qrcode
import base58
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def import_privKey(privkey_filepath,password=None):
    if password==None:
        password = getpass.getpass("Input password for decrypting keyfile: ")
    password=password.encode()
    f=open(privkey_filepath,'rb')
    decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
    privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
    return privKey

def import_pubKey(pubkey_filepath):
    f=open(pubkey_filepath,'rb')
    privKey=eth_keys.keys.PublicKey(f.read())
    return privKey

def import_addr(addr_filepath):
    return open(addr_filepath,'rb').read().decode()

# Comenzar con Claves Existentes (Start with Existing Keys)

Acceder a las claves generadas en el cuaderno 5
***
Access the keys generated in the cuaderno 5

## Comenzar con Claves en Google Drive (Start with Keys on Google Drive)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp drive/MyDrive/cinv/llaves.zip llaves.zip 
! unzip llaves.zip


## Comenzar con Claves en la Disco Local
## (Start with Keys on Local Drive)

In [ ]:
#from google.colab import files
#fu=files.upload();

In [ ]:
#! unzip llaves.zip

In [ ]:
#! ls llaves

# Import Private Key, Public Key and Address

In [ ]:
privKey=import_privKey('llaves/mi_prv.enc','')
pub=import_pubKey('llaves/mi_pub.bin').to_hex()
addr=import_addr('llaves/mi_addr.bin')

In [ ]:
pub

In [ ]:
addr

In [ ]:
from IPython.display import Image
qr=Image('llaves/mi_addr.png')
display(qr)

# Import 

# Connect to Colegio Message Spreadsheet

## Decrypt Credentials

$\color{white}{\text{c0leg101nv1s1ble}}$

Use la contraseña de arriba para descifrar las credenciales
***
Use password above to decrypt credentials


In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py c0leg101nv1s1ble Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

## Autenticar y Conectarse a la Google Sheet de Mensajes (Authenticate and Connect to Message Google Sheet)

Autenticarse usando credenciales descifradas para la API de Google Drive.\
Use la biblioteca `gspread` para leer y escribir en una hoja de cálculo compartida.
***
Authenticate using decrypted credentials for Google Drive API.\
Use `gspread` library to read and write to shared spreadsheet.

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# Leer y Escribir en la Google Sheet de Mensajes (Read and Write to Messages Google Sheet)

## Leer Google Sheet (Read Google Sheet)

In [ ]:
sheets.worksheets()

In [ ]:
sheet4=sheets.worksheets()[4]
sheet4.row_values(1)

In [ ]:
#sheet4.clear()
#sheet4.append_row(["name", "address","pubkey"])

## Ingresar el Nombre (Input Name)

In [ ]:
my_name=input('input name')

## Leer Nombres Publicados (Read Posted Names)

In [ ]:
peeps=sheet4.col_values(1)[1:]
peeps

## Escribir en la Google Sheet para Publicar Nombre (Write to Google Sheet to Publish Name)

Si el nombre está en la lista, escriba la dirección en la celda correspondiente de la hoja\
Si el nombre no está presente, agregue el nombre y la dirección a la hoja
***
If the name is in the list then write address to the appropriate cell of sheet\
If name is not present append name and address to sheet

In [88]:
pub

'0x7c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5'

In [89]:
addr

'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX'

In [90]:
if not (my_name in peeps) :
  sheet4.append_row([my_name,addr,pub])
else:
  sheet4.update_cell(row=peeps.index(my_name)+2,col=2,value=addr)
  sheet4.update_cell(row=peeps.index(my_name)+2,col=3,value=pub)

## Leer Direcciones (Read Addresses)

In [111]:
import pandas as pd

sheet4=sheets.worksheets()[4]
dataframe4 = pd.DataFrame(sheet4.get_all_records())
dataframe4

,name,address,pubkey
0,DrDoeg,D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX,0x7c88e9a4df6e9f45656c10bf66f28e28be235a15b648...
1,Laura,DJBkcc9zShCCWPTvPuLUiHQegNPbmMipEH,0x2478459160f8e4563ad84a1faa998158c2e47b3c8eac...


## Construir Lista de los Direcciones de Destinatario (Construct List of Recipient Addresses) 

In [112]:
addresses = dataframe4.address.tolist()
addresses

['D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX', 'DJBkcc9zShCCWPTvPuLUiHQegNPbmMipEH']

In [113]:
pubs = dataframe4.pubkey.tolist()
pubs = [ '04'+pb[2:] for pb in pubs ]
pubs

['047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5',
 '042478459160f8e4563ad84a1faa998158c2e47b3c8eac04654ca30f892d4c873db08339cabb0f5c775f1f798919ab4ce5772666a88ea39cf811b73c95d28ee9f9']

# Enviar Direcciones de Criptomonedas con Transacción de Financiación (Send Addresses Cryptocurrency with Funding Transaction)

## Importar 'cryptos' y Crear un Objeto de Dogecoin (Import `cryptos` and Create Dogecoin Object)

In [104]:
import cryptos

In [109]:
doge=cryptos.Doge()
doge.script_magicbyte=22

In [110]:
multi_tup=doge.mk_multsig_address(pubs,2)
multi_tup

('5241047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c541042478459160f8e4563ad84a1faa998158c2e47b3c8eac04654ca30f892d4c873db08339cabb0f5c775f1f798919ab4ce5772666a88ea39cf811b73c95d28ee9f952ae',
 '9vvCKFQYC4r4C3ZUsNZqyxkxV4YgFEiNDg')

## Las Entradas (Inputs)

### Identificar la Dirección de Financiación (Identify Funding Address)

In [114]:
banco=addresses[0]
banco

'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX'

### Determinar las Entradas de UTXO (Determine UTXO Inputs)

***
***
Los UTXO son salidas de transacciones no gastadas. Comenzamos revisando la cadena de bloques para identificar las monedas que la dirección de financiación ha recibido pero que aún no ha gastado. Estos son UTXO.
***
UTXOs are Unspent Transaction Outputs. We start by checking the blockchain to identify coins the funding address has received but has not yet spent. These are UTXOs. 
***
***

Cada UTXO contiene (each UTXO contains):
- valor (value)
- transacción (transaction)
  - hash (hash)
  - índice (index)

In [115]:
#UTXOS
unspents=doge.unspent(banco)
unspents

[{'output': '98d01787a9f8722c154402e18e59c4251d4583e36067a7013826bc3343fb13af:0',
  'value': 4200000000}]

### Satoshis

El valor se informa en unidades de Satoshis/Dogetoshis. Un Satoshi es un $\frac{1}{100,000,000}$ de una moneda.
***
The value is reported in units of Satoshis/Dogetoshis. A Satoshi is $\frac{1}{100,000,000}$ of a coin.

### Sumar Salidas No Gastadas (Sum Unspent Outputs)

In [116]:
unspent_total=sum([unspent['value'] for unspent in unspents])
unspent_total

4200000000

## Salidas (Outputs)

### Calcular Valores de Salida (Calculate Output Values)

- `gift`: es la cantidad entregada a cada participante\
(the amount given to each participant)
- `gift_total`: la cantidad total entregada\
(the total amount given)
- `mining_fee`: cantidad entregada a los mineros\
(amount given to miners)
- `change`: cantidad sobrante que el remitente enviará a una dirección de cambio\
(the left over amount that the sender will send to a change address)

In [117]:
gift=10_00_000_000
gift_total = gift * len(addresses)
mining_fee = 1_00_000_000
change = unspent_total - gift_total - mining_fee 

### Construir Lista de Salidas (Construct Output List)
***
***
Crear salidas que incluyan:
***
Create outputs including:
***
***
- `gift` regalo a cada participante\
 (`gift` to each participant)
- `change` enviado de vuelta a `banco` direccion\
(`change` sent back to `banco` address)

In [118]:
outputs=[{'value': gift, 'address': address} for address in addresses]
outputs.append({'value': change, 'address': banco})
outputs

[{'address': 'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX', 'value': 1000000000},
 {'address': 'DJBkcc9zShCCWPTvPuLUiHQegNPbmMipEH', 'value': 1000000000},
 {'address': 'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX', 'value': 2100000000}]


## Hacer Transacción de Financiamiento (Make Funding Transaction)

In [119]:
fund_txn=doge.mktx(unspents,outputs )
fund_txn

{'ins': [{'amount': 4200000000,
   'outpoint': {'hash': '98d01787a9f8722c154402e18e59c4251d4583e36067a7013826bc3343fb13af',
    'index': 0},
   'script': '',
   'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': '76a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac',
   'value': 1000000000},
  {'script': '76a9148f1a4020cc7a25b8adfe2a04c7a8fe72c8a7665a88ac',
   'value': 1000000000},
  {'script': '76a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac',
   'value': 2100000000}],
 'version': 1}

## Firmar Transacción de Financiamiento (Sign Funding Transaction)

In [120]:
doge.signall(fund_txn,privKey.to_hex()[2:])

'0100000001af13fb4333bc263801a76760e383451d25c4598ee10244152c72f8a98717d098000000008b483045022100dff5ab367016ca22e580f4cbad53002debaea04876cfd9146afe3f7c80f4216b02206d9025334c67d36a50c6e0142fc8020d433e373227249fc3748b36dfe8e903990141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5ffffffff0300ca9a3b000000001976a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac00ca9a3b000000001976a9148f1a4020cc7a25b8adfe2a04c7a8fe72c8a7665a88ac00752b7d000000001976a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac00000000'

In [121]:
fund_txn

{'ins': [{'amount': 4200000000,
   'outpoint': {'hash': '98d01787a9f8722c154402e18e59c4251d4583e36067a7013826bc3343fb13af',
    'index': 0},
   'script': '483045022100dff5ab367016ca22e580f4cbad53002debaea04876cfd9146afe3f7c80f4216b02206d9025334c67d36a50c6e0142fc8020d433e373227249fc3748b36dfe8e903990141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5',
   'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': '76a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac',
   'value': 1000000000},
  {'script': '76a9148f1a4020cc7a25b8adfe2a04c7a8fe72c8a7665a88ac',
   'value': 1000000000},
  {'script': '76a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac',
   'value': 2100000000}],
 'version': 1}

In [122]:
fund_txn_hex=cryptos.serialize(fund_txn)
fund_txn_hex

'0100000001af13fb4333bc263801a76760e383451d25c4598ee10244152c72f8a98717d098000000008b483045022100dff5ab367016ca22e580f4cbad53002debaea04876cfd9146afe3f7c80f4216b02206d9025334c67d36a50c6e0142fc8020d433e373227249fc3748b36dfe8e903990141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5ffffffff0300ca9a3b000000001976a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac00ca9a3b000000001976a9148f1a4020cc7a25b8adfe2a04c7a8fe72c8a7665a88ac00752b7d000000001976a914144739367df0ff8d1c61d03704298d49cf93ef3f88ac00000000'

## Comprobar el Tamaño en Kilobytes de la Transacción (Check Kilobyte Size of Transaction)

https://chain.so/DOGE

In [123]:
kb=len(fund_txn_hex)/2000
kb

0.292

In [124]:

mining_fee/1E8/kb

3.4246575342465757

## Transmitir la Transacción (Broadcast Transaction)

In [125]:
broadcast_fund_txn=doge.pushtx(fund_txn_hex)
broadcast_fund_txn

{'data': {'network': 'DOGE',
  'txid': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94'},
 'status': 'success'}

## Inspeccionar la Transacción (Inspect Transaction)

In [128]:
inspect_fund_txn=doge.fetchtx( broadcast_fund_txn['data']['txid'])
inspect_fund_txn

{'blockhash': '580f88a5f3218d13ef9c54d746ca5eb44c19b0e3bd143af4e4d1925ba6020976',
 'confirmations': 1,
 'inputs': [{'address': 'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX',
   'from_output': {'output_no': 0,
    'txid': '98d01787a9f8722c154402e18e59c4251d4583e36067a7013826bc3343fb13af'},
   'input_no': 0,
   'script': '3045022100dff5ab367016ca22e580f4cbad53002debaea04876cfd9146afe3f7c80f4216b02206d9025334c67d36a50c6e0142fc8020d433e373227249fc3748b36dfe8e9039901 047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5',
   'sequence': 4294967295,
   'type': 'pubkeyhash',
   'value': '42.00000000',
   'witness': None}],
 'locktime': 0,
 'network': 'DOGE',
 'network_fee': '1.00000000',
 'outputs': [{'address': 'D6zKNnkupqRbkB9p5rwix8QiobQWJazjyX',
   'output_no': 0,
   'script': 'OP_DUP OP_HASH160 144739367df0ff8d1c61d03704298d49cf93ef3f OP_EQUALVERIFY OP_CHECKSIG',
   'type': 'pubkeyhash',
   'value': '10.00000000'},
  {'a

# Devolver la Criptomoneda (Return Coins)

## Construir la Transacción de Devolver (Construct Return Transaction)

### Entradas (Inputs)

In [135]:
mi_unspents=doge.unspent(addr)[:1]
mi_unspents


[{'output': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94:0',
  'value': 1000000000}]

In [137]:
all_unspents=[ doge.unspent(addy)[0] for addy in addresses ]
all_unspents

[{'output': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94:0',
  'value': 1000000000},
 {'output': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94:1',
  'value': 1000000000}]

In [138]:
all_unspent_total=sum([unspent['value'] for unspent in all_unspents])
all_unspent_total

2000000000

### Salidas (Outputs)

In [139]:
mi_mining_fee = 1_00_000_000
disponible = all_unspent_total - mi_mining_fee
disponible

1900000000

In [142]:
multi_addr=multi_tup[1]
multi_addr

'9vvCKFQYC4r4C3ZUsNZqyxkxV4YgFEiNDg'

In [143]:
destinos=[{'value': disponible, 'address': multi_addr} ]
destinos

[{'address': '9vvCKFQYC4r4C3ZUsNZqyxkxV4YgFEiNDg', 'value': 1900000000}]

## Pare !! Ladrón !! (Stop Thief)

Para robar las monedas en lugar de devolverlas, ejecute las celdas **`Ladrón`** al final del cuaderno.
***
To steal the coins instead of returning them, run the **`Ladrón`** cells at the end of the notebook.

### Construcción (Constuction)

In [144]:
volver_txn=doge.mktx(all_unspents, destinos)
volver_txn

{'ins': [{'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 0},
   'script': '',
   'sequence': 4294967295},
  {'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 1},
   'script': '',
   'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': 'a9143128856aefebe150d940ad7e0c76b4f3f88218ca87',
   'value': 1900000000}],
 'version': 1}

## Firmar y Serializar (Sign and Serialize)

In [145]:
doge.sign(volver_txn, 0,privKey.to_hex()[2:])

{'ins': [{'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 0},
   'script': '483045022100bea26913cac0406069c02d11844543ed17fcf946c0f5cc3006e59d1e3249a5ea0220268d8f4ce6e7cf4319c376b0ada4a581331e7fb67b75d2524f059a5a515d19950141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5',
   'sequence': 4294967295},
  {'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 1},
   'script': '',
   'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': 'a9143128856aefebe150d940ad7e0c76b4f3f88218ca87',
   'value': 1900000000}],
 'version': 1}

In [150]:
my_sig=volver_txn['ins'][0]['script']
my_sig

'483045022100bea26913cac0406069c02d11844543ed17fcf946c0f5cc3006e59d1e3249a5ea0220268d8f4ce6e7cf4319c376b0ada4a581331e7fb67b75d2524f059a5a515d19950141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5'

In [151]:
sig1='4730440220562629b6e48055d3822696007ee30b0b73b5374f8284a925c8fe362ada2cc08c0220061654549c667330e88231abd01d3da8eae7f3659ed1d2eb3a348e55e3a811280141042478459160f8e4563ad84a1faa998158c2e47b3c8eac04654ca30f892d4c873db08339cabb0f5c775f1f798919ab4ce5772666a88ea39cf811b73c95d28ee9f9'

In [154]:
volver_txn['ins'][1]['script']=sig1
volver_txn

{'ins': [{'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 0},
   'script': '483045022100bea26913cac0406069c02d11844543ed17fcf946c0f5cc3006e59d1e3249a5ea0220268d8f4ce6e7cf4319c376b0ada4a581331e7fb67b75d2524f059a5a515d19950141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5',
   'sequence': 4294967295},
  {'amount': 1000000000,
   'outpoint': {'hash': '2e06d1607e15d8be9e7892f56bbad77f647af30538c3f2ab88835ed9200fcd94',
    'index': 1},
   'script': '4730440220562629b6e48055d3822696007ee30b0b73b5374f8284a925c8fe362ada2cc08c0220061654549c667330e88231abd01d3da8eae7f3659ed1d2eb3a348e55e3a811280141042478459160f8e4563ad84a1faa998158c2e47b3c8eac04654ca30f892d4c873db08339cabb0f5c775f1f798919ab4ce5772666a88ea39cf811b73c95d28ee9f9',
   'sequence': 4294967295}],
 'locktime': 0,
 'outs': [{'script': 'a9143128856aefebe150d940ad7e0c76b4f3f88218

In [155]:
volver_txn_hex=cryptos.serialize(volver_txn)
volver_txn_hex

'010000000294cd0f20d95e8388abf2c33805f37a647fd7ba6bf592789ebed8157e60d1062e000000008b483045022100bea26913cac0406069c02d11844543ed17fcf946c0f5cc3006e59d1e3249a5ea0220268d8f4ce6e7cf4319c376b0ada4a581331e7fb67b75d2524f059a5a515d19950141047c88e9a4df6e9f45656c10bf66f28e28be235a15b64820b254f1b9eb2738314e6769f5c94da0c7640ffe76dcffca053b07a0804cd53a1c51ad03bfe0133ce8c5ffffffff94cd0f20d95e8388abf2c33805f37a647fd7ba6bf592789ebed8157e60d1062e010000008a4730440220562629b6e48055d3822696007ee30b0b73b5374f8284a925c8fe362ada2cc08c0220061654549c667330e88231abd01d3da8eae7f3659ed1d2eb3a348e55e3a811280141042478459160f8e4563ad84a1faa998158c2e47b3c8eac04654ca30f892d4c873db08339cabb0f5c775f1f798919ab4ce5772666a88ea39cf811b73c95d28ee9f9ffffffff0100b33f710000000017a9143128856aefebe150d940ad7e0c76b4f3f88218ca8700000000'

## Transmisión (Broadcast)

In [156]:
broadcast_volver_txn=doge.pushtx(volver_txn_hex)
broadcast_volver_txn

{'data': {'network': 'DOGE',
  'txid': '9acd7bee16c465f059ff75d1e3773b1daa90ff5e1c59bfe83e6dd97f196f0e7b'},
 'status': 'success'}